# Automating Experiments



Resource management algorithms often take user-specified parameters such as thresholds and flow control flags. When this happens, it is typically necessary to run several simulations varying the input values to find the best set of parameters.

This notebook demonstrates how we can create automated test plans in EdgeSimPy.

Let's start by loading some dependencies:

In [1]:
try:
    # Importing EdgeSimPy components
    from edge_sim_py import *
    import networkx as nx
    import msgpack

    # Importing Matplotlib, Pandas, and NumPy for logs parsing and visualization
    import matplotlib.pyplot as plt
    import pandas as pd
    import numpy as np

except ModuleNotFoundError:
    # Downloading EdgeSimPy binaries from GitHub (the "-q" parameter suppresses Pip's output. You check the full logs by removing it)
    %pip install -q git+https://github.com/EdgeSimPy/EdgeSimPy.git

    # Downloading Pandas, NumPy, and Matplotlib (these are not directly used here, but they can be useful for logs parsing and visualization)
    %pip install -q pandas==2.2.2
    %pip install -q numpy==1.26.4
    %pip install -q matplotlib==3.8.0

    # Importing EdgeSimPy components and its built-in libraries (NetworkX and MessagePack)
    from edge_sim_py import *
    import networkx as nx
    import msgpack

    # Importing Matplotlib, Pandas, and NumPy for logs parsing and visualization
    import matplotlib.pyplot as plt
    import pandas as pd
    import numpy as np

# Importing Python's built-in modules
import itertools


For conciseness, this notebook doesn't focus on configuring simulations on EdgeSimPy. Instead, we will use the scenario described in [this notebook](https://github.com/EdgeSimPy/edgesimpy-tutorials/blob/master/notebooks/creating-placement-algorithm.ipynb), in which a First-Fit algorithm defines which edge servers should host services within the infrastructure.

The following two cells present the source code for the First-Fit algorithm and the simulation's stopping criterion (i.e., halting the simulation when all services are provisioned).

In [2]:
def my_algorithm(parameters):
    print(f"parameters: {parameters}")
    # We can always call the 'all()' method to get a list with all created instances of a given class
    for service in Service.all():
        # We don't want to migrate services are are already being migrated
        if service.server == None and not service.being_provisioned:

            # Let's iterate over the list of edge servers to find a suitable host for our service
            for edge_server in EdgeServer.all():

                # We must check if the edge server has enough resources to host the service
                if edge_server.has_capacity_to_host(service=service):

                    # Start provisioning the service in the edge server
                    service.provision(target_server=edge_server)

                    # After start migrating the service we can move on to the next service
                    break


def stopping_criterion(model: object):
    # Defining a variable that will help us to count the number of services successfully provisioned within the infrastructure
    provisioned_services = 0
    
    # Iterating over the list of services to count the number of services provisioned within the infrastructure
    for service in Service.all():

        # Initially, services are not hosted by any server (i.e., their "server" attribute is None).
        # Once that value changes, we know that it has been successfully provisioned inside an edge server.
        if service.server != None:
            provisioned_services += 1
    
    # As EdgeSimPy will halt the simulation whenever this function returns True, its output will be a boolean expression
    # that checks if the number of provisioned services equals to the number of services spawned in our simulation
    return provisioned_services == Service.count()

Please notice that resource management algorithms in EdgeSimPy take an optional argument called "parameters". In this notebook, we will leverage such an argument, passing some parameters to our First-Fit algorithm. For simplicity's sake, we will create two arbitrary parameters called "a" and "b". Each parameter has some valid values that we want to evaluate.

Our goal is to create an automated execution plan that runs our First-Fit algorithm with all combinations of values of "a" and "b".

In [3]:
parameters = {
    "a": [1, 2, 3],
    "b": [100, 200, 3],
}

Now that we have defined the parameters that will compose the test configurations, we can leverage Python's built-in features like `itertools.product()` to generate different test specifications (i.e., combinations of parameters).

The cell below executes simulations for each specification, instructing EdgeSimPy to store the logs in separate directories to facilitate result analysis.

In [4]:
for combination in itertools.product(*parameters.values()):
    specification = {}
    specification_name = ''
    for parameter_id, parameter_value in enumerate(combination):
        parameter_name = list(parameters.keys())[parameter_id]
        
        # Formatting the logs directory name according to the current specification's parameters
        specification_name += f";{parameter_name}={parameter_value}" if parameter_id else f"{parameter_name}={parameter_value}"

        # Building a dictionary from the current specification
        specification[parameter_name] = parameter_value
    
    # Creating a Simulator object with the current parameters specification and its logs directory
    simulator = Simulator(
        dump_interval=5,
        tick_duration=1,
        tick_unit="seconds",
        stopping_criterion=stopping_criterion,
        resource_management_algorithm=my_algorithm,
        resource_management_algorithm_parameters=specification,
        logs_directory=specification_name
    )

    # Loading a sample dataset from GitHub
    simulator.initialize(input_file="https://raw.githubusercontent.com/EdgeSimPy/edgesimpy-tutorials/master/datasets/sample_dataset2.json")

    # Executing the simulation
    print(f"==== Running Simulation with Specification: {specification} ====")
    simulator.run_model()
    

==== Running Simulation with Specification: {'a': 1, 'b': 100, 'current_step': 1} ====
parameters: {'a': 1, 'b': 100, 'current_step': 1}
parameters: {'a': 1, 'b': 100, 'current_step': 2}
parameters: {'a': 1, 'b': 100, 'current_step': 3}
parameters: {'a': 1, 'b': 100, 'current_step': 4}
parameters: {'a': 1, 'b': 100, 'current_step': 5}
parameters: {'a': 1, 'b': 100, 'current_step': 6}
parameters: {'a': 1, 'b': 100, 'current_step': 7}
parameters: {'a': 1, 'b': 100, 'current_step': 8}
==== Running Simulation with Specification: {'a': 1, 'b': 200, 'current_step': 1} ====
parameters: {'a': 1, 'b': 200, 'current_step': 1}
parameters: {'a': 1, 'b': 200, 'current_step': 2}
parameters: {'a': 1, 'b': 200, 'current_step': 3}
parameters: {'a': 1, 'b': 200, 'current_step': 4}
parameters: {'a': 1, 'b': 200, 'current_step': 5}
parameters: {'a': 1, 'b': 200, 'current_step': 6}
parameters: {'a': 1, 'b': 200, 'current_step': 7}
parameters: {'a': 1, 'b': 200, 'current_step': 8}
==== Running Simulation wi